# AoC - Day 5

## Part 1 - Problem Statement

The Thermal Environment Supervision Terminal (TEST) starts by running a diagnostic program (your puzzle input). The TEST diagnostic program will run on your existing Intcode computer after a few modifications:

First, you'll need to add two new instructions:

Opcode 3 takes a single integer as input and saves it to the position given by its only parameter. For example, the instruction 3,50 would take an input value and store it at address 50.
Opcode 4 outputs the value of its only parameter. For example, the instruction 4,50 would output the value at address 50.
Programs that use these instructions will come with documentation that explains what should be connected to the input and output. The program 3,0,4,0,99 outputs whatever it gets as input, then halts.

Second, you'll need to add support for parameter modes:

Each parameter of an instruction is handled based on its parameter mode. Right now, your ship computer already understands parameter mode 0, position mode, which causes the parameter to be interpreted as a position - if the parameter is 50, its value is the value stored at address 50 in memory. Until now, all parameters have been in position mode.

Now, your ship computer will also need to handle parameters in mode 1, immediate mode. In immediate mode, a parameter is interpreted as a value - if the parameter is 50, its value is simply 50.

Parameter modes are stored in the same value as the instruction's opcode. The opcode is a two-digit number based only on the ones and tens digit of the value, that is, the opcode is the rightmost two digits of the first value in an instruction. Parameter modes are single digits, one per parameter, read right-to-left from the opcode: the first parameter's mode is in the hundreds digit, the second parameter's mode is in the thousands digit, the third parameter's mode is in the ten-thousands digit, and so on. Any missing modes are 0.

Parameters that an instruction writes to will never be in immediate mode.

Finally, some notes:

It is important to remember that the instruction pointer should increase by the number of values in the instruction after the instruction finishes. Because of the new instructions, this amount is no longer always 4.
Integers can be negative: 1101,100,-1,4,0 is a valid program (find 100 + -1, store the result in position 4).

## Solution

In [1]:
from functools import reduce
import operator

In [2]:
def read_program(program):
    return [int(token) for token in program.split(',')]

In [3]:
def build_memory(program):
    return dict(enumerate(read_program(program)))

In [4]:
operation = {
    '01': 'add',
    '02': 'multiply',
    '03': 'input',
    '04': 'output'
}

In [5]:
mode = {
    '0': 'position',
    '1': 'immediate'
}

In [6]:
def parse_opcode(opcode):
    opcode = f'{opcode:05}'
    instruction = opcode[3:]
    
    op = operation[instruction]
    if op in ('add', 'multiply'):
        return {
            'operation': op,
            'mode_param1': mode[opcode[2]],
            'mode_param2': mode[opcode[1]],
            'mode_param3': 'target',
            'ip_shift': 4
        }
    elif op in ('input', 'output'):
        return {
            'operation': op,
            'mode_param1': 'target' if mode[opcode[2]] != 'immediate' else 'immediate',
            'ip_shift': 2
        }
    else:
        assert False

In [7]:
def get_param_value(memory, mode, value):
    return memory[value] if mode == 'position' else value

In [8]:
def parse_instruction(ip, memory):
    opcode = memory[ip]
    instruction = parse_opcode(opcode)
    
    operation = instruction['operation']
    if operation in ('add', 'multiply'):
        param1 = get_param_value(memory, instruction['mode_param1'], memory[ip+1])
        param2 = get_param_value(memory, instruction['mode_param2'], memory[ip+2])
        param3 = get_param_value(memory, instruction['mode_param3'], memory[ip+3])
        return {'operation': operation,
                'param1': param1,
                'param2': param2,
                'param3': param3,
                'next_ip': ip + instruction['ip_shift']}
    elif operation in ('input', 'output'):
        param1 = get_param_value(memory, instruction['mode_param1'], memory[ip+1])
        return {'operation': operation,
                'param1': param1,
                'mode_param1': instruction['mode_param1'],
                'next_ip': ip + instruction['ip_shift']}
    else:
        assert False

In [9]:
def execute_instruction(ip, memory):
    instruction = parse_instruction(ip, memory)
    
    if instruction['operation'] == 'add':
        memory[instruction['param3']] = instruction['param1'] + instruction['param2']
    elif instruction['operation'] == 'multiply':
        memory[instruction['param3']] = instruction['param1'] * instruction['param2']
    elif instruction['operation'] == 'input':
        memory[instruction['param1']] = int(input())
    elif instruction['operation'] == 'output':
        print(instruction['param1'] if instruction['mode_param1'] == 'immediate' else memory[instruction['param1']])
    else:
        assert False
    
    return {
        'ip': instruction['next_ip'],
        'memory': memory
    }

In [10]:
def run_program(program):
    memory = build_memory(program)
    ip = 0
    
    while memory[ip] != 99:
        ip, memory = operator.itemgetter('ip', 'memory')(execute_instruction(ip, memory))

## Tests

In [11]:
assert read_program('1002,4,3,4,33') == [1002, 4, 3, 4, 33]

In [12]:
assert build_memory('1002,4,3,4,33') == {0: 1002, 1: 4, 2: 3, 3: 4, 4: 33}

In [13]:
assert parse_opcode(1002) == {
    'operation': 'multiply',
    'mode_param1': 'position',
    'mode_param2': 'immediate',
    'mode_param3': 'target',
    'ip_shift': 4
}

In [14]:
assert get_param_value(memory=build_memory('1002,4,3,4,33'),
                       mode='immediate',
                       value=50) == 50
assert get_param_value(memory=build_memory('1002,4,3,4,33'),
                       mode='position',
                       value=4) == 33
assert get_param_value(memory=build_memory('1004,4,3,4,33'),
                       mode='target',
                       value=4) == 4

In [15]:
assert (parse_instruction(ip=0,
                          memory=build_memory('1002,4,3,4,33'))
        == {'operation': 'multiply',
            'param1': 33,
            'param2': 3,
            'param3': 4,
            'next_ip': 4})

In [16]:
assert (execute_instruction(ip=0,
                            memory=build_memory('1002,4,3,4,33'))
        == {'memory': {0: 1002, 1: 4, 2: 3, 3: 4, 4: 99},
            'ip': 4})

## Data

In [17]:
program = '3,225,1,225,6,6,1100,1,238,225,104,0,1102,45,16,225,2,65,191,224,1001,224,-3172,224,4,224,102,8,223,223,1001,224,5,224,1,223,224,223,1102,90,55,225,101,77,143,224,101,-127,224,224,4,224,102,8,223,223,1001,224,7,224,1,223,224,223,1102,52,6,225,1101,65,90,225,1102,75,58,225,1102,53,17,224,1001,224,-901,224,4,224,1002,223,8,223,1001,224,3,224,1,224,223,223,1002,69,79,224,1001,224,-5135,224,4,224,1002,223,8,223,1001,224,5,224,1,224,223,223,102,48,40,224,1001,224,-2640,224,4,224,102,8,223,223,1001,224,1,224,1,224,223,223,1101,50,22,225,1001,218,29,224,101,-119,224,224,4,224,102,8,223,223,1001,224,2,224,1,223,224,223,1101,48,19,224,1001,224,-67,224,4,224,102,8,223,223,1001,224,6,224,1,223,224,223,1101,61,77,225,1,13,74,224,1001,224,-103,224,4,224,1002,223,8,223,101,3,224,224,1,224,223,223,1102,28,90,225,4,223,99,0,0,0,677,0,0,0,0,0,0,0,0,0,0,0,1105,0,99999,1105,227,247,1105,1,99999,1005,227,99999,1005,0,256,1105,1,99999,1106,227,99999,1106,0,265,1105,1,99999,1006,0,99999,1006,227,274,1105,1,99999,1105,1,280,1105,1,99999,1,225,225,225,1101,294,0,0,105,1,0,1105,1,99999,1106,0,300,1105,1,99999,1,225,225,225,1101,314,0,0,106,0,0,1105,1,99999,7,226,677,224,102,2,223,223,1005,224,329,1001,223,1,223,8,226,677,224,1002,223,2,223,1005,224,344,101,1,223,223,8,226,226,224,1002,223,2,223,1006,224,359,101,1,223,223,1008,677,226,224,1002,223,2,223,1005,224,374,1001,223,1,223,108,677,677,224,1002,223,2,223,1005,224,389,1001,223,1,223,1107,226,677,224,1002,223,2,223,1006,224,404,101,1,223,223,1008,226,226,224,102,2,223,223,1006,224,419,1001,223,1,223,7,677,226,224,1002,223,2,223,1005,224,434,101,1,223,223,1108,226,226,224,1002,223,2,223,1005,224,449,101,1,223,223,7,226,226,224,102,2,223,223,1005,224,464,101,1,223,223,108,677,226,224,102,2,223,223,1005,224,479,1001,223,1,223,1007,677,226,224,1002,223,2,223,1006,224,494,1001,223,1,223,1007,677,677,224,1002,223,2,223,1006,224,509,1001,223,1,223,107,677,677,224,1002,223,2,223,1005,224,524,101,1,223,223,1108,226,677,224,102,2,223,223,1006,224,539,1001,223,1,223,8,677,226,224,102,2,223,223,1005,224,554,101,1,223,223,1007,226,226,224,102,2,223,223,1006,224,569,1001,223,1,223,107,677,226,224,102,2,223,223,1005,224,584,1001,223,1,223,108,226,226,224,102,2,223,223,1006,224,599,1001,223,1,223,107,226,226,224,1002,223,2,223,1006,224,614,1001,223,1,223,1108,677,226,224,1002,223,2,223,1005,224,629,1001,223,1,223,1107,677,677,224,102,2,223,223,1005,224,644,1001,223,1,223,1008,677,677,224,102,2,223,223,1005,224,659,101,1,223,223,1107,677,226,224,1002,223,2,223,1006,224,674,101,1,223,223,4,223,99,226'

In [18]:
run_program(program)

1
0
0
0
0
0
0
0
0
0
12440243


## Part 2 - Problem Statement

Your computer is only missing a few opcodes:

* Opcode 5 is jump-if-true: if the first parameter is non-zero, it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing.
* Opcode 6 is jump-if-false: if the first parameter is zero, it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing.
* Opcode 7 is less than: if the first parameter is less than the second parameter, it stores 1 in the position given by the third parameter. Otherwise, it stores 0.
* Opcode 8 is equals: if the first parameter is equal to the second parameter, it stores 1 in the position given by the third parameter. Otherwise, it stores 0.


Like all instructions, these instructions need to support parameter modes as described above.

Normally, after an instruction is finished, the instruction pointer increases by the number of values in that instruction. However, if the instruction modifies the instruction pointer, that value is used and the instruction pointer is not automatically increased.

## Solution

In [19]:
operation = {
    '01': 'add',
    '02': 'multiply',
    '03': 'input',
    '04': 'output',
    '05': 'jump-if-true',
    '06': 'jump-if-false',
    '07': 'less-than',
    '08': 'equals'
}

In [25]:
def parse_opcode(opcode):
    opcode = f'{opcode:05}'
    instruction = opcode[3:]
    
    op = operation[instruction]
    if op in ('add', 'multiply', 'less-than', 'equals'):
        return {
            'operation': op,
            'mode_param1': mode[opcode[2]],
            'mode_param2': mode[opcode[1]],
            'mode_param3': 'target',
            'ip_shift': 4
        }
    elif op in ('input', 'output'):
        return {
            'operation': op,
            'mode_param1': 'target' if mode[opcode[2]] != 'immediate' else 'immediate',
            'ip_shift': 2
        }
    elif op in ('jump-if-true', 'jump-if-false'):
        return {
            'operation': op,
            'mode_param1': mode[opcode[2]],
            'mode_param2': mode[opcode[1]],
            'ip_shift': 'jump'
        }
    else:
        assert False

In [26]:
def parse_instruction(ip, memory):
    opcode = memory[ip]
    instruction = parse_opcode(opcode)
    operation = instruction['operation']
    
    if operation in ('add', 'multiply', 'less-than', 'equals'):
        param1 = get_param_value(memory, instruction['mode_param1'], memory[ip+1])
        param2 = get_param_value(memory, instruction['mode_param2'], memory[ip+2])
        param3 = get_param_value(memory, instruction['mode_param3'], memory[ip+3])
        return {'operation': operation,
                'param1': param1,
                'param2': param2,
                'param3': param3,
                'next_ip': ip + instruction['ip_shift']}
    elif operation in ('input', 'output'):
        param1 = get_param_value(memory, instruction['mode_param1'], memory[ip+1])
        return {'operation': operation,
                'param1': param1,
                'mode_param1': instruction['mode_param1'],
                'next_ip': ip + instruction['ip_shift']}
    elif operation in ('jump-if-true', 'jump-if-false'):
        param1 = get_param_value(memory, instruction['mode_param1'], memory[ip+1])
        param2 = get_param_value(memory, instruction['mode_param2'], memory[ip+2])
        return {'operation': operation,
                'param1': param1,
                'param2': param2,
                'next_ip': 'jump'}
    else:
        assert False

In [27]:
def execute_instruction(ip, memory, input_function, output):
    instruction = parse_instruction(ip, memory)
    
    if instruction['operation'] == 'add':
        memory[instruction['param3']] = instruction['param1'] + instruction['param2']
    elif instruction['operation'] == 'multiply':
        memory[instruction['param3']] = instruction['param1'] * instruction['param2']
    elif instruction['operation'] == 'input':
        memory[instruction['param1']] = int(input_function())
    elif instruction['operation'] == 'output':
        output.append(instruction['param1'] if instruction['mode_param1'] == 'immediate' else memory[instruction['param1']])
    elif instruction['operation'] == 'jump-if-true':
        instruction['next_ip'] = instruction['param2'] if instruction['param1'] != 0 else ip + 3
    elif instruction['operation'] == 'jump-if-false':
        instruction['next_ip'] = instruction['param2'] if instruction['param1'] == 0 else ip + 3
    elif instruction['operation'] == 'less-than':
        memory[instruction['param3']] = 1 if instruction['param1'] < instruction['param2'] else 0
    elif instruction['operation'] == 'equals':
        memory[instruction['param3']] = 1 if instruction['param1'] == instruction['param2'] else 0
    else:
        assert False
    
    return {
        'ip': instruction['next_ip'],
        'memory': memory,
        'output': output
    }

In [23]:
def run_program(program, input_function=input):
    memory = build_memory(program)
    ip = 0
    
    output = []
    while memory[ip] != 99:
        ip, memory, output = operator.itemgetter('ip', 'memory', 'output')(execute_instruction(ip, memory, input_function, output))
    
    return output

## Tests

In [28]:
assert run_program('3,9,8,9,10,9,4,9,99,-1,8', lambda : 8) == [1]
assert run_program('3,9,8,9,10,9,4,9,99,-1,8', lambda : 7) == [0]

In [29]:
assert run_program('3,9,7,9,10,9,4,9,99,-1,8', lambda : 8) == [0]
assert run_program('3,9,7,9,10,9,4,9,99,-1,8', lambda : 7) == [1]

In [30]:
assert run_program('3,3,1108,-1,8,3,4,3,99', lambda : 8) == [1]
assert run_program('3,3,1108,-1,8,3,4,3,99', lambda : 7) == [0]

In [31]:
assert run_program('3,3,1107,-1,8,3,4,3,99', lambda : 8) == [0]
assert run_program('3,3,1107,-1,8,3,4,3,99', lambda : 7) == [1]

In [32]:
assert run_program('3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9', lambda : 0) == [0]
assert run_program('3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9', lambda : 10) == [1]

In [33]:
assert run_program('3,3,1105,-1,9,1101,0,0,12,4,12,99,1', lambda : 0) == [0]
assert run_program('3,3,1105,-1,9,1101,0,0,12,4,12,99,1', lambda : 10) == [1]

In [34]:
test_program = '''
3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99'''

assert run_program(test_program, lambda : 7) == [999]
assert run_program(test_program, lambda : 8) == [1000]
assert run_program(test_program, lambda : 9) == [1001]

In [35]:
run_program(program)

5


[15486302]